In [1]:
import transformers

In [ ]:
import pdfplumber
pdf_path="./uploads/AFFAIRE A.A.K. c. TÜRKiYE.pdf"
def extract_text_from_pdf(pdf_path):
    text = ''
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()
    return text
df = extract_text_from_pdf(pdf_path)
print(df)

In [3]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model_name = "t5-small"  # Adjust to the model you choose
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
import re
import os
!pip install frontend
!pip install pymupdf
import fitz  # PyMuPDF library for PDF extraction

from transformers import T5ForConditionalGeneration, T5Tokenizer
from langdetect import detect  # Import the language detection function

In [5]:
print(transformers.__version__)

4.34.1


In [6]:
def preprocess_text(text):
    # Remove special characters
    text = re.sub(r'[^\w\s,.()\[\]{}"\'<>/\\|:;_\-?!@#%&*+=\^]', '', text)

    # Convert text to lowercase
    text = text.lower()

    return text

In [7]:
# !pip install langdetect
from langdetect import detect


In [8]:
def translate_pdf(pdf_file_path):
    # Extract text from the PDF file using PyMuPDF
    doc = fitz.open(pdf_file_path)
    text = ""
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text += page.get_text()

    # Preprocess the extracted text
    preprocessed_text = preprocess_text(text)

    # Build the translation input
    translation_input = f"translate {preprocessed_text}"

    # Tokenize the input text
    input_ids = tokenizer.encode(translation_input, return_tensors="pt")

    # Generate the translation
    translation_ids = model.generate(input_ids, max_length=512, num_return_sequences=1, early_stopping=True)

    # Decode and return the translation
    translation = tokenizer.decode(translation_ids[0], skip_special_tokens=True)

    return translation

# Path to the PDF file you want to translate
pdf_file_path = "./uploads/AFFAIRE C.P. ET M.N. c. FRANCE.pdf"

# Translate the PDF file
translated_text = translate_pdf(pdf_file_path)
# print(translated_text)

Token indices sequence length is longer than the specified maximum sequence length for this model (15287 > 512). Running this sequence through the model will result in indexing errors
C:\Users\enigma\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\generation\configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


l'enfant a été élu à l'occasion de la première enquête de la cour de l'année précédente, et a été élu à l'occasion de la première enquête de l'année précédente, et a été élu à l'occasion de la première enquête de l'année précédente, et a été élu à l'occasion de la première enquête de


In [13]:
import fitz  # PyMuPDF
from transformers import pipeline
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

# Initialize the translation pipeline with source and target languages
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-fr", device=0, framework="pt", use_fast=True, tokenizer="Helsinki-NLP/opus-mt-en-fr")

# Input PDF file path
pdf_file_path = "./uploads/AFFAIRE C.P. ET M.N. c. FRANCE.pdf"

# Output PDF file path
output_pdf_path = "translated_outputAFFAIRE C.P. ET M.N. c. FRANCE.pdf"

# Initialize a PDF canvas for the output file
c = canvas.Canvas(output_pdf_path, pagesize=letter)

In [21]:
import fitz
import transformers
from transformers import TranslationPipeline


In [ ]:
# translator = TranslationPipeline(
#     model="Helsinki-NLP/opus-mt-en-fr",
#     task="translation",
# )


In [32]:
import fitz
from transformers import pipeline
# Function to translate text with a specified max_length
# def translate_text(text, translator, max_length=2000):
#     # Translate the text from English to French with the specified max_length
#     translated_text = translator(text, max_length=max_length)
#     return translated_text
# translated_text = translate_text(text, translator, max_length=2000)  # Adjust max_length as needed

# Function to create a new PDF document for translated text
def create_translated_pdf(translated_texts, page_sizes):
    translated_pdf = fitz.open()
    for text, page_size in zip(translated_texts, page_sizes):
        page = translated_pdf.new_page(width=page_size[0], height=page_size[1])
        page.insert_text(text, fontsize=12)
    return translated_pdf
# Function to translate and append pages to the output PDF
# Function to translate and append pages to the output PDF
def translate_and_append(pdf_document, start_page, end_page, translator, max_length):
    translated_pages = []  # Store translated pages

    for page_num in range(start_page, end_page):
        page = pdf_document[page_num]
        text = page.get_text("text")

        # Translate the text from English to French with the specified max_length
        translated_text = translator(text, max_length=max_length)

        # Create a new page in the output PDF with the same dimensions as the original
        page_width = int(page.rect.width)
        page_height = int(page.rect.height)
        new_page = output_pdf_document.new_page(page_width, page_height)

        # Add the translated text to the new page
        new_page.insert_text(text=translated_text, fontsize=12)

        translated_pages.append(new_page)

    return translated_pages


# Set the max_length value for translation
max_length = 800  # Adjust this value as needed

# Translate and append pages to the output PDF
translated_pages = translate_and_append(pdf_document, start_page, end_page, translator, max_length)

# Add the translated pages to the output PDF
for page in translated_pages:
    output_pdf_document.insert_pdf(page)

# Open the input PDF file
pdf_file_path = "./uploads/AFFAIRE C.P. ET M.N. c. FRANCE.pdf"
pdf_document = fitz.open(pdf_file_path)
total_pages = len(pdf_document)

# Initialize translation pipeline
translator = pipeline("translation_en_to_fr")

# Set the number of pages to process at a time
chunk_size = 10

output_pdf_path = "translated_outputAFFAIRE C.P. ET M.N. c. FRANCE.pdf"
output_pdf_document = fitz.open()

# Translate and append pages to the output PDF in chunks
start_page = 0
while start_page < total_pages:
    end_page = min(start_page + chunk_size, total_pages)
    translated_texts = []

    for page_num in range(start_page, end_page):
        page = pdf_document.load_page(page_num)
        text = page.get_text("text")
        translated_text = translate_text(text, translator)
        translated_texts.append(translated_text)

    # Get page sizes
    page_sizes = [(int(page.rect.width), int(page.rect.height)) for page in pdf_document.pages(start_page, end_page)]

    # Create a new PDF document for translated text
    translated_pdf = create_translated_pdf(translated_texts, page_sizes)
    output_pdf_document.insert_pdf(translated_pdf)

    start_page = end_page

# Save the output PDF file
output_pdf_document.save(output_pdf_path)
output_pdf_document.close()

# Close the input PDF file
pdf_document.close()

print(f"Translation complete. Translated PDF saved at: {output_pdf_path}")


RuntimeError: cannot insert page beyond end of page tree

In [41]:
import fitz
from transformers import pipeline
from fitz.utils import Point

# Function to create a new PDF document for a single page of translated text
def create_translated_page(translated_text, page_size):
    translated_pdf = fitz.open()
    page = translated_pdf.new_page(width=page_size[0], height=page_size[1])
    page.insert_text(Point(50, 50), translated_text, fontsize=12)  # Adjust the position as needed
    return translated_pdf

# Set the max_length value for translation
max_length = 800  # Adjust this value as needed

# Function to translate a single page and append it to the output PDF
def translate_and_append_single_page(page, translator):
    text = page.get_text("text")

    # Translate the text from English to French
    translated_text = translator(text, max_length=2000)
    print(translated_text)

    # Get the width and height of the page
    page_width = int(page.rect.width)
    page_height = int(page.rect.height)

    # Create a new PDF document for the translated page
    translated_pdf = create_translated_page(translated_text, (page_width, page_height))

    return translated_pdf

# Open the input PDF file
pdf_file_path = "./uploads/AFFAIRE C.P. ET M.N. c. FRANCE.pdf"
pdf_document = fitz.open(pdf_file_path)
total_pages = len(pdf_document)

# Initialize translation pipeline
translator = pipeline("translation_en_to_fr")

# Set the number of pages to process at a time
chunk_size = 1

output_pdf_path = "translated_outputAFFAIRE C.P. ET M.N. c. FRANCE.pdf"

# Create a new output PDF document
output_pdf_document = fitz.open()

# Translate and append pages to the output PDF one at a time
for page_num in range(total_pages):
    page = pdf_document[page_num]

    # Translate and append the current page
    translated_pdf = translate_and_append_single_page(page, translator)

    # Add the translated page to the output PDF
    output_pdf_document.insert_pdf(translated_pdf)
    # outputFile.append()

# Save the output PDF file
output_pdf_document.save(output_pdf_path)
output_pdf_document.close()

# Close the input PDF file
pdf_document.close()

print(f"Translation complete. Translated PDF saved at: {output_pdf_path}")


No model was supplied, defaulted to t5-base and revision 686f1db (https://huggingface.co/t5-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'translation_text': 'CINQUIME SECTION AFFAIRE C.P. et M.N. c. FRANCE (Requêtes nos 56513/17 et 56515/17) ARRT Art 8 • Vie privée et familiale • Refus des juridictions internes d’examiner l’action du requérant, affirmant être le père biologique d’un enfant, visant à contester la paternité légalement'}]


Token indices sequence length is longer than the specified maximum sequence length for this model (644 > 512). Running this sequence through the model will result in indexing errors


[{'translation_text': '   '}]
[{'translation_text': 'ARRT C.P. et M.N. c. FRANCE 1 En l’affaire C.P. et M.N. c. France, La Cour européenne des droits de l’homme (cinquième section), siégeant en une chambre composée de : Georges Ravarani, président, Carlo Ranzoni, Mrti Mits, Stéphanie Mourou-Vikström, Mara El'}]
[{'translation_text': 'ARRT C.P. et M.N. c. FRANCE 2 5. Au début du mois de mars 2012, la requérante quitta son ancien compagnon et conclut un pacte civil de solidarité (PACS) avec le requérant, le 14 mars 2012. 6. Le 12 décembre 2012, la requérante saisit le juge aux affaires familiales (JAF) aux fins de faire fixer les mesures relatives aux deux'}]
[{'translation_text': 'ARRT C.P. et M.N. c. FRANCE 3 « (...) nul, à l’exception du ministère public, ne peut contester la filiation lorsque la possession d’état conforme à une reconnaissance a duré au moins cinq ans depuis la naissance ou la reconnaissance, si elle a été faite ultérieurement (...). » 13. Par ailleurs, il constata, d

In [39]:
print(translated_pdf)

Document(<new PDF, doc# 63>)


In [ ]:
# import re

# def preprocess_text(text):
#     # Remove page numbers


#     # Remove special characters 
#     text = re.sub(r'[^\w\s,.()\[\]{}"\'<>/\\|:;_\-?!@#%&*+=\^]', '', text)

#     # Convert text to lowercase
#     text = text.lower()

#     return text

# # Assuming you have a function called extract_text_from_pdf that extracts text from a PDF
# # Example usage:
# preprocessed_text = preprocess_text(df)
# print(preprocessed_text)


In [37]:
from transformers import pipeline
t5 = pipeline(model="t5-base", tokenizer="t5-base")


C:\Users\enigma\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\models\t5\tokenization_t5_fast.py:158: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
C:\Users\enigma\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\pipelines\__init__.py:1013: UserWarning: "translation" task was used, instead of "translation_XX_to_YY", defaulting to "tran

In [40]:
def translate(text):
    return t5(text)


In [44]:
translate("hello there, general kenobi you are a bold one")

[{'translation_text': 'Hallo da, allgemeine kenobi Sie sind ein mutiger'}]

In [28]:
# from transformers import T5ForConditionalGeneration, T5Tokenizer

# model_name = "t5-base"
# model = T5ForConditionalGeneration.from_pretrained(model_name)
# tokenizer = T5Tokenizer.from_pretrained(model_name)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [34]:

# # Your long Turkish text
# turkish_text = preprocessed_text

# # Split the text into segments
# max_seq_length = 512
# segments = [turkish_text[i:i+max_seq_length] for i in range(0, len(turkish_text), max_seq_length)]

# # Translate each segment
# translated_text_segments = []


In [35]:
# segment = segments[0]
# input_ids = tokenizer.encode("translate Turkish to English: " + segment, return_tensors="pt")
# translation = model.generate(input_ids, max_length=150, num_beams=4, no_repeat_ngram_size=2, early_stopping=True)
# translated_text = tokenizer.decode(translation[0], skip_special_tokens=True)
# translated_text_segments.append(translated_text)

In [36]:

# # Combine the translated segments
# translated_text = " ".join(translated_text_segments)

# # print("Turkish Text: ", turkish_text)
# print("English Translation: ", translated_text)


English Translation:  türkiye (requête no 56578/11) arrêt art 8 vie privée mise sous tutelle judiciaire de la requérante, à lissue dune procédure ayant conclu quelle souffrait dun trouble mental entravant sa capacité dagir existence de garanties effectives dans la procédure interne pour prévenir les abus en veillant  ce que les droits et les interess of the respondent soient pris è compte participation de cette
